# A Simple Fuzzer

In [1]:
from typing import Dict, List, Tuple, Union, Any

In [2]:
import random

In [3]:
def fuzzer(max_length: int = 100, char_start: int = 32, char_range: int = 32) -> str:
    '''
    A string of up to max_length characters
    int the range [char_start, char_start + char_range) 
    '''
    str_len = random.randrange(0, max_length + 1)
    out = ''
    for i in range(0, str_len):
        out += chr(random.randrange(char_start, char_start + char_range))
    return out

In [4]:
fuzzer()

'?'

In [5]:
fuzzer(1000, ord('a'), 26)  # produce massive a~z data string

'eymosxexnqqhrwedpppuqdegemehrzfqhngxpuipgtwknlegvhmcdlhahrhsbrsnpendpwpkfvzprshgsevhzeiarswcrijyczjympjcebeydvwpkqhhuegzsssbfzgqdddofnlvlfeaxnfbteloffytxofpbvxfpmfkjlhhhghkuakndqqlgoagsnocxzfbepjqrnteztjkytmnodjxyrwolhlecessudicdzvaqppnbxixploihkyfrcrrwbvcbkffmiithetbftwfotzzzhpqigdptqbceyflwbebhnyadlahspxitepzbjylvgdgdqjsxarlotanvmojcxzrwdrlsodyukjcwpkfpfbytbgwykvlgjvkquzsedvbnlwchzrxrgsztzxjsgzknstjtufocabimcihikuoyqqkjafvfndddhcmgzkeklfkgjcwjmfbpdtvacdrwsvbhsvbyrfjoiggl'

In [6]:
fuzzer(200, ord('0'), 10)

'9414389087412708858022202689350160341146482154392850960165449466712763037787542972336424404567067708228136751221209530060392805546464308111815'

# Fuzzing External Programs

## Creating Input Files

In [7]:
import os
import tempfile

In [8]:
basename = 'input.txt'
tempdir = tempfile.mkdtemp()
FILE = os.path.join(tempdir, basename)
print(FILE)

C:\Users\hxie\AppData\Local\Temp\tmpqx_kdn6s\input.txt


In [9]:
data = fuzzer()
with open(FILE, 'w') as f:
    f.write(data)

In [10]:
contents = open(FILE).read()
print(contents)
assert(contents == data)

*+-2%#!=3%);<??'-,*!((%+"706"4/!3?0"> ;($ ,1%5,,16-;%5';:5 ,"2<?'6>7&8'/&6=2((-=


## Invoking External Programs

In [11]:
import os
import subprocess

bc is a math command in Unix

In [13]:
program = 'C:\\Windows\\System32\\calc.exe'
with open(FILE, 'w') as f:
    f.write('2 + 2\n')
result = subprocess.run([program, FILE],
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)
print(result)

CompletedProcess(args=['C:\\Windows\\System32\\calc.exe', 'C:\\Users\\hxie\\AppData\\Local\\Temp\\tmprtb2bv2v\\input.txt'], returncode=0, stdout='', stderr='')


In [14]:
result.stdout

''

In [15]:
result.stderr

''

In [16]:
result.returncode

0

# A Fuzzing Architecture

## Runner Classes

In [12]:
class Runner:
    '''
    Base class for testing inputs
    '''
    PASS = 'PASS'
    FAIL = 'FAIL'
    UNRESOLVED = 'UNRESOLVED'

    def __init__(self) -> None:
        pass
        
    def run(self, inp: str) -> Any:
        return(inp, Runner.UNRESOLVED)

In [13]:
class PrintRunner(Runner):
    def run(self, inp) -> Any:
        print(inp)
        return(inp, Runner.UNRESOLVED)

In [14]:
p = PrintRunner()
(result, outcome) = p.run('Some input')

Some input


In [15]:
result

'Some input'

In [16]:
outcome

'UNRESOLVED'

## Fuzzer Classes

In [17]:
Outcome = str

In [18]:
class Fuzzer:
    '''Base class for fuzzers'''
    def __init__(self) -> None:
        pass

    def fuzz(self) -> str:
        return ''
    
    def run(self, runner: Runner = Runner()) \
        -> Tuple[subprocess.CompletedProcess, Outcome]:
        return runner.run(self.fuzz())
    
    def runs(self, runner: Runner = PrintRunner(), trails: int = 10) \
        -> List[Tuple[subprocess.CompletedProcess, Outcome]]:
        return [self.run(runner) for i in range(trails)]

In [20]:
class RandomFuzzer(Fuzzer):
    """Produce random inputs."""

    def __init__(self, min_length: int = 10, max_length: int = 100,
                 char_start: int = 32, char_range: int = 32) -> None:
        """Produce strings of `min_length` to `max_length` characters
           in the range [`char_start`, `char_start` + `char_range`)"""
        self.min_length = min_length
        self.max_length = max_length
        self.char_start = char_start
        self.char_range = char_range

    def fuzz(self) -> str:
        string_length = random.randrange(self.min_length, self.max_length + 1)
        out = ""
        for i in range(0, string_length):
            out += chr(random.randrange(self.char_start,
                                        self.char_start + self.char_range))
        return out

In [21]:
random_fuzzer = RandomFuzzer(min_length=20, max_length=20)
for i in range(10):
    print(random_fuzzer.fuzz())

+'."&-'(%15<2$ >1=6%
617+../*7(!.:>7399>7
)4"6'<'7-&/8-$.0<1<8
6&1+*/34).0"#!76<1%9
>4=4%-0/.58(62-;,%%#
 .$'*1*>.7)'/+ >101:
)>07+(!(7.43&,31(+%+
*7<9"& :>#;97%'!/9()
44-0)''8#>7=#-#(5?1"
!$3:!2/(.9'6/, =70>8
